In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc

sc.settings.set_figure_params(figsize=(4, 4))
import pandas as pd
import numpy as np
import itertools
import scanpy_helpers as sh

In [3]:
adata_all = sc.read_h5ad("../../data/30_merge_adata/adata_scvi.h5ad")
adata_malignant_b = sc.read_h5ad(
    "../../data/40_cluster_analysis/adata_malignant_b_cells.h5ad"
)
artifact_dir = "../../data/50_de_analysis/pseudobulk"

In [4]:
adata = adata_all[adata_all.obs["cell_type"] == "malignant B cell", :].copy()

In [5]:
adata.obs

,patient,timepoint,sample,n_genes_by_counts,total_counts,total_counts_mito,pct_counts_mito,n_counts,n_genes,patient_id,sex,age,ethnicity,response,leiden,leiden_scvi,cell_type
cell_id,,,,,,,,,,,,,,,,,
646419_0,P1,T1,HCL_P1_T1,2757,11779.0,2893.0,24.560659,11779.0,2757,P1,female,85,caucasian,short_term,10,0,malignant B cell
825469_0,P1,T1,HCL_P1_T1,2607,10887.0,3520.0,32.332142,10887.0,2607,P1,female,85,caucasian,short_term,10,1,malignant B cell
818696_0,P1,T1,HCL_P1_T1,2604,10103.0,3070.0,30.387014,10103.0,2604,P1,female,85,caucasian,short_term,10,4,malignant B cell
41552_0,P1,T1,HCL_P1_T1,2383,9685.0,2996.0,30.934435,9685.0,2383,P1,female,85,caucasian,short_term,10,4,malignant B cell
467785_0,P1,T1,HCL_P1_T1,2561,10090.0,2453.0,24.311199,10090.0,2561,P1,female,85,caucasian,short_term,10,0,malignant B cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820086_11,P5,T0,HCL_P5_T0,793,2050.0,777.0,37.902439,2050.0,793,P5,male,56,caucasian,long_term,2,2,malignant B cell
133321_11,P5,T0,HCL_P5_T0,780,2036.0,800.0,39.292732,2036.0,780,P5,male,56,caucasian,long_term,2,5,malignant B cell
222098_11,P5,T0,HCL_P5_T0,750,2130.0,723.0,33.943661,2130.0,750,P5,male,56,caucasian,long_term,2,2,malignant B cell


In [6]:
adata.obs.groupby(
    ["patient", "timepoint", "sample", "response"], observed=True
).size().reset_index(name="size").sort_values(["timepoint", "patient"])

,patient,timepoint,sample,response,size
7,P2,T0,HCL_P2_T0,short_term,3350
4,P3,T0,HCL_P3_T0,short_term,1093
8,P4,T0,HCL_P4_T0,long_term,3782
10,P5,T0,HCL_P5_T0,long_term,5211
9,P6,T0,HCL_P6_T0,long_term,4186
0,P1,T1,HCL_P1_T1,short_term,383
5,P2,T1,HCL_P2_T1,short_term,2329
2,P3,T1,HCL_P3_T1,short_term,6352
1,P1,T2,HCL_P1_T2,short_term,37
6,P2,T2,HCL_P2_T2,short_term,2374


## Comparison between short- and long-term responders
* Only T0
* All timepoints

In [7]:
sh.pseudobulk.write_pseudobulk(
    sh.pseudobulk.pseudobulk(
        adata, groupby=["patient", "response", "sex", "age"], layer="raw_counts"
    ),
    f"{artifact_dir}/bulk_response_all_timepoints",
)

In [8]:
sh.pseudobulk.write_pseudobulk(
    sh.pseudobulk.pseudobulk(
        adata[adata.obs["timepoint"] == "T0", :],
        groupby=["patient", "response", "sex", "age"],
        layer="raw_counts",
    ),
    f"{artifact_dir}/bulk_response_t0",
)

## DE analysis of timepoints (pseudobulk, only P2-3)

In [9]:
tmp_adata = adata[adata.obs["patient"].isin(["P2", "P3"]), :]
tmp_adata.obs["timepoint"] = [
    "pre-treatment" if t == "T0" else "post-treatment"
    for t in tmp_adata.obs["timepoint"]
]

Trying to set attribute `.obs` of view, copying.


In [10]:
sh.pseudobulk.write_pseudobulk(
    sh.pseudobulk.pseudobulk(
        tmp_adata, groupby=["patient", "timepoint", "sex", "age"], layer="raw_counts"
    ),
    f"{artifact_dir}/bulk_timepoints",
)

## Comparison of JUN+ cluster vs rest of malignant B cells

In [11]:
adata_malignant_b.obs

,patient,timepoint,sample,n_genes_by_counts,total_counts,total_counts_mito,pct_counts_mito,n_counts,n_genes,patient_id,sex,age,ethnicity,response,leiden,leiden_scvi,cell_type,cell_phenotype
cell_id,,,,,,,,,,,,,,,,,,
646419_0,P1,T1,HCL_P1_T1,2757,11779.0,2893.0,24.560659,11779.0,2757,P1,female,85,caucasian,short_term,10,0,malignant B cell,malignant_b
825469_0,P1,T1,HCL_P1_T1,2607,10887.0,3520.0,32.332142,10887.0,2607,P1,female,85,caucasian,short_term,10,1,malignant B cell,malignant_b
818696_0,P1,T1,HCL_P1_T1,2604,10103.0,3070.0,30.387014,10103.0,2604,P1,female,85,caucasian,short_term,10,4,malignant B cell,malignant_b
41552_0,P1,T1,HCL_P1_T1,2383,9685.0,2996.0,30.934435,9685.0,2383,P1,female,85,caucasian,short_term,10,4,malignant B cell,malignant_b
467785_0,P1,T1,HCL_P1_T1,2561,10090.0,2453.0,24.311199,10090.0,2561,P1,female,85,caucasian,short_term,10,0,malignant B cell,malignant_b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820086_11,P5,T0,HCL_P5_T0,793,2050.0,777.0,37.902439,2050.0,793,P5,male,56,caucasian,long_term,2,2,malignant B cell,malignant_b
133321_11,P5,T0,HCL_P5_T0,780,2036.0,800.0,39.292732,2036.0,780,P5,male,56,caucasian,long_term,2,5,malignant B cell,fos_malignant_b
222098_11,P5,T0,HCL_P5_T0,750,2130.0,723.0,33.943661,2130.0,750,P5,male,56,caucasian,long_term,2,2,malignant B cell,malignant_b


In [12]:
sh.pseudobulk.write_pseudobulk(
    sh.pseudobulk.pseudobulk(
        adata_malignant_b, groupby=["patient", "cell_phenotype"], layer="raw_counts"
    ),
    f"{artifact_dir}/bulk_fos_jun_vs_rest",
)

## Comparison of healthy and malignant B cells

In [13]:
adata_healthy_malignant = adata_all[
    adata_all.obs["cell_type"].isin(["malignant B cell", "healthy B cell"]), :
].copy()
adata_healthy_malignant.obs["cell_type"] = (
    adata_healthy_malignant.obs["cell_type"].str.lower().str.replace(" ", "_")
)
sh.pseudobulk.write_pseudobulk(
    sh.pseudobulk.pseudobulk(
        adata_healthy_malignant,
        groupby=["cell_type", "patient"],
        layer="raw_counts",
    ),
    f"{artifact_dir}/bulk_healthy_malignant",
)